<a href="https://colab.research.google.com/github/younesabdolmalaky/Solar-Power-Plant-Data/blob/main/Solar_Power_Plant_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import catboost
from sklearn.preprocessing import StandardScaler

In [37]:
df = pd.read_csv('/content/Solar Power Plant Data.csv')
df.head()

,Date-Hour(NMT),WindSpeed,Sunshine,AirPressure,Radiation,AirTemperature,RelativeAirHumidity,SystemProduction
0,01.01.2017-00:00,0.6,0,1003.8,-7.4,0.1,97,0.0
1,01.01.2017-01:00,1.7,0,1003.5,-7.4,-0.2,98,0.0
2,01.01.2017-02:00,0.6,0,1003.4,-6.7,-1.2,99,0.0
3,01.01.2017-03:00,2.4,0,1003.3,-7.2,-1.3,99,0.0
4,01.01.2017-04:00,4.0,0,1003.1,-6.3,3.6,67,0.0


In [38]:
df['Date-Hour(NMT)'] = pd.to_datetime(df['Date-Hour(NMT)'], format='%d.%m.%Y-%H:%M')
df['Date'] = df['Date-Hour(NMT)'].dt.date
df['Hour'] = df['Date-Hour(NMT)'].dt.hour

In [39]:
def convert_to_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Autumn"

df['Date'] = pd.to_datetime(df['Date'])
df['Season'] = df['Date'].apply(convert_to_season)


In [40]:
def convert_to_time_of_day(hour):
    if hour >= 11 and hour < 15:
        return "day"
    else:
        return "Night"

df['TimeOfDay'] = df['Hour'].apply(convert_to_time_of_day)

In [41]:
df = pd.get_dummies(df, columns = ['Season' , 'TimeOfDay'])
df = df.drop(['Date' , 'Date-Hour(NMT)'] , axis = 1)
df.head()

,WindSpeed,Sunshine,AirPressure,Radiation,AirTemperature,RelativeAirHumidity,SystemProduction,Hour,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,TimeOfDay_Night,TimeOfDay_day
0,0.6,0,1003.8,-7.4,0.1,97,0.0,0,0,0,0,1,1,0
1,1.7,0,1003.5,-7.4,-0.2,98,0.0,1,0,0,0,1,1,0
2,0.6,0,1003.4,-6.7,-1.2,99,0.0,2,0,0,0,1,1,0
3,2.4,0,1003.3,-7.2,-1.3,99,0.0,3,0,0,0,1,1,0
4,4.0,0,1003.1,-6.3,3.6,67,0.0,4,0,0,0,1,1,0


In [50]:
X = df.drop(['SystemProduction'],axis = 1)
y = df['SystemProduction']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [55]:
model = catboost.CatBoostRegressor(eval_metric='MAE',iterations=20000)
model.fit(X_train, y_train , eval_set = (X_test , y_test))

Streaming output truncated to the last 5000 lines.
15005:	learn: 76.0768338	test: 237.2177726	best: 237.2164923 (14999)	total: 38.5s	remaining: 12.8s
15006:	learn: 76.0743061	test: 237.2157237	best: 237.2157237 (15006)	total: 38.5s	remaining: 12.8s
15007:	learn: 76.0719401	test: 237.2137269	best: 237.2137269 (15007)	total: 38.5s	remaining: 12.8s
15008:	learn: 76.0672159	test: 237.2162885	best: 237.2137269 (15007)	total: 38.5s	remaining: 12.8s
15009:	learn: 76.0640014	test: 237.2143780	best: 237.2137269 (15007)	total: 38.5s	remaining: 12.8s
15010:	learn: 76.0619597	test: 237.2156305	best: 237.2137269 (15007)	total: 38.5s	remaining: 12.8s
15011:	learn: 76.0567355	test: 237.2168255	best: 237.2137269 (15007)	total: 38.5s	remaining: 12.8s
15012:	learn: 76.0498702	test: 237.2178622	best: 237.2137269 (15007)	total: 38.5s	remaining: 12.8s
15013:	learn: 76.0429437	test: 237.2170602	best: 237.2137269 (15007)	total: 38.5s	remaining: 12.8s
15014:	learn: 76.0379987	test: 237.2173971	best: 237.21372

In [56]:
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2

0.8490497172401736